Importando Pacotes

In [18]:
# !pip install -r requirements.txt

In [3]:
import pandas as pd
import requests
import json

Configurações e variaveis gerais do código

Documentação API: https://www.api-football.com/documentation-v3

In [4]:
#Exibir totas as colunas no pandas
pd.set_option('display.max_columns', None)

base_url = "https://v3.football.api-sports.io"
headers = {
  'x-rapidapi-key': '75a1d666035ab70c3321c2884333be2b',
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

0 - Variaveis da Coleta

In [149]:
season = 2023
id_liga = 71 #Brasileirao Serie A
limite_request = 50 #Limitando o numero de chamadas devido caracteristica da API Free

path_output = "data/raw/brasileirao_2023/"

1 - Get partidas do campeonato

In [10]:
url_partidas = f"{base_url}/fixtures?timezone=America/Sao_Paulo&league={id_liga}&season={season}"
response_partidas = requests.get(url_partidas, headers=headers).json()

In [11]:
df_partidas = pd.json_normalize(response_partidas['response'])
df_partidas['fixture.date'] = pd.to_datetime(df_partidas['fixture.date'])

In [61]:
df_partidas.head()

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.logo,league.flag,league.season,league.round,teams.home.id,teams.home.name,teams.home.logo,teams.home.winner,teams.away.id,teams.away.name,teams.away.logo,teams.away.winner,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away
0,1005649,"Rodrigo Jose Pereira de Lima, Brazil",America/Sao_Paulo,2023-04-16 16:00:00-03:00,1681671600,1.681672e+09,1.681675e+09,NaN,Estádio do Maracanã,Rio de Janeiro,Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,127,Flamengo,https://media-4.api-sports.io/football/teams/1...,True,147,Coritiba,https://media-4.api-sports.io/football/teams/1...,False,3.0,0.0,1.0,0.0,3.0,0.0,None,None,None,None
1,1005650,"Ramon Abatti Abel, Brazil",America/Sao_Paulo,2023-04-15 18:30:00-03:00,1681594200,1.681594e+09,1.681598e+09,218.0,Estádio Nilton Santos,Rio de Janeiro,Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,120,Botafogo,https://media-4.api-sports.io/football/teams/1...,True,126,Sao Paulo,https://media-4.api-sports.io/football/teams/1...,False,2.0,1.0,1.0,1.0,2.0,1.0,None,None,None,None
2,1005651,"Paulo Cesar Zanovelli da Silva, Brazil",America/Sao_Paulo,2023-04-15 16:00:00-03:00,1681585200,1.681585e+09,1.681589e+09,258.0,Allianz Parque,"São Paulo, São Paulo",Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,121,Palmeiras,https://media-4.api-sports.io/football/teams/1...,True,1193,Cuiaba,https://media-4.api-sports.io/football/teams/1...,False,2.0,1.0,1.0,1.0,2.0,1.0,None,None,None,None
3,1005652,"Anderson Daronco, Brazil",America/Sao_Paulo,2023-04-16 16:00:00-03:00,1681671600,1.681672e+09,1.681675e+09,11531.0,Neo Química Arena,"São Paulo, São Paulo",Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,131,Corinthians,https://media-4.api-sports.io/football/teams/1...,True,135,Cruzeiro,https://media-4.api-sports.io/football/teams/1...,False,2.0,1.0,0.0,0.0,2.0,1.0,None,None,None,None
4,1005653,"Maguielson Lima Barbosa, Brazil",America/Sao_Paulo,2023-04-15 18:30:00-03:00,1681594200,1.681594e+09,1.681598e+09,220.0,Estádio Nabi Abi Chedid,"Bragança Paulista, São Paulo",Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,794,RB Bragantino,https://media-4.api-sports.io/football/teams/7...,True,118,Bahia,https://media-4.api-sports.io/football/teams/1...,False,2.0,1.0,0.0,1.0,2.0,1.0,None,None,None,None


Output Arquivo

In [31]:
df_partidas.to_parquet(path_output + 'partidas/partidas.parquet')

2 - Get Estatisticas das partidas do campeonato

In [145]:
#Todas as partidas com status Finalizado (FT)
todas_partidas = set(df_partidas[df_partidas['fixture.status.short'] == 'FT']['fixture.id'])

#Partidas já coletadas
try:
    partidas_coletadas = set(pd.read_parquet(f'{path_output}/estatistica_partida/')['fixture.id'])
except:
    partidas_coletadas = set()

#Gera lista de partidas novas à serem coletadas, limitando ao máximo permitido pela API
id_partidas_coleta = list(todas_partidas - partidas_coletadas)[:limite_request]


print('qtd partidas a coletar: ', len(id_partidas_coleta))

qtd partidas a coletar:  50


Consulta e Output arquivos

In [147]:
for id in id_partidas_coleta:
    url_statistics = f"{base_url}/fixtures/statistics?fixture={id}"
    response_statistics = requests.get(url_statistics, headers=headers).json()

    df_estatistica = pd.json_normalize(response_statistics['response'],
                                       record_path="statistics",
                                       meta=[["team", "id"], ["team", "name"]]
                                       ).astype('str')
    df_estatistica['fixture.id'] = id
    
    output_file = f'{path_output}/partidas_estatistica/partida_{id}.parquet'
    df_estatistica.to_parquet(output_file)
    print(output_file)

raw/brasileirao_2023//estatistica_partida/partida_1005649.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005650.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005651.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005652.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005653.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005654.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005655.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005656.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005657.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005658.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005659.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005660.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005661.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005662.parquet
raw/brasileirao_2023//estatistica_partida/partida_1005663.parquet
raw/brasil

Teste Leitura Arquivos

In [151]:
df_teste_partidas = pd.read_parquet(path_output + 'partidas/partidas.parquet')
df_teste_partidas.head()

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.logo,league.flag,league.season,league.round,teams.home.id,teams.home.name,teams.home.logo,teams.home.winner,teams.away.id,teams.away.name,teams.away.logo,teams.away.winner,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away
0,1005649,"Rodrigo Jose Pereira de Lima, Brazil",America/Sao_Paulo,2023-04-16 16:00:00-03:00,1681671600,1.681672e+09,1.681675e+09,NaN,Estádio do Maracanã,Rio de Janeiro,Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,127,Flamengo,https://media-4.api-sports.io/football/teams/1...,True,147,Coritiba,https://media-4.api-sports.io/football/teams/1...,False,3.0,0.0,1.0,0.0,3.0,0.0,None,None,None,None
1,1005650,"Ramon Abatti Abel, Brazil",America/Sao_Paulo,2023-04-15 18:30:00-03:00,1681594200,1.681594e+09,1.681598e+09,218.0,Estádio Nilton Santos,Rio de Janeiro,Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,120,Botafogo,https://media-4.api-sports.io/football/teams/1...,True,126,Sao Paulo,https://media-4.api-sports.io/football/teams/1...,False,2.0,1.0,1.0,1.0,2.0,1.0,None,None,None,None
2,1005651,"Paulo Cesar Zanovelli da Silva, Brazil",America/Sao_Paulo,2023-04-15 16:00:00-03:00,1681585200,1.681585e+09,1.681589e+09,258.0,Allianz Parque,"São Paulo, São Paulo",Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,121,Palmeiras,https://media-4.api-sports.io/football/teams/1...,True,1193,Cuiaba,https://media-4.api-sports.io/football/teams/1...,False,2.0,1.0,1.0,1.0,2.0,1.0,None,None,None,None
3,1005652,"Anderson Daronco, Brazil",America/Sao_Paulo,2023-04-16 16:00:00-03:00,1681671600,1.681672e+09,1.681675e+09,11531.0,Neo Química Arena,"São Paulo, São Paulo",Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,131,Corinthians,https://media-4.api-sports.io/football/teams/1...,True,135,Cruzeiro,https://media-4.api-sports.io/football/teams/1...,False,2.0,1.0,0.0,0.0,2.0,1.0,None,None,None,None
4,1005653,"Maguielson Lima Barbosa, Brazil",America/Sao_Paulo,2023-04-15 18:30:00-03:00,1681594200,1.681594e+09,1.681598e+09,220.0,Estádio Nabi Abi Chedid,"Bragança Paulista, São Paulo",Match Finished,FT,90.0,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,794,RB Bragantino,https://media-4.api-sports.io/football/teams/7...,True,118,Bahia,https://media-4.api-sports.io/football/teams/1...,False,2.0,1.0,0.0,1.0,2.0,1.0,None,None,None,None


In [160]:
df_teste_estatistica = pd.read_parquet(path_output + 'partidas_estatistica/')
df_teste_estatistica.head(20)

,type,value,team.id,team.name,fixture.id
0,Shots on Goal,7,127,Flamengo,1005649
1,Shots off Goal,5,127,Flamengo,1005649
2,Total Shots,15,127,Flamengo,1005649
3,Blocked Shots,3,127,Flamengo,1005649
4,Shots insidebox,5,127,Flamengo,1005649
5,Shots outsidebox,10,127,Flamengo,1005649
6,Fouls,20,127,Flamengo,1005649
7,Corner Kicks,6,127,Flamengo,1005649
8,Offsides,1,127,Flamengo,1005649
9,Ball Possession,59%,127,Flamengo,1005649
